In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import glob
from collections import OrderedDict

In [ ]:
#directories = glob.glob('.experiments/end-effector_tuning/*/*num_hiddens-2*/result*.pth')
directories = glob.glob('.experiments/end-effector_tune_beta/*/*/result*.pth')
#load each 
results = {}
seed = 0
is_one_model = False
for pth in directories:
    act = pth.split('/')[-2].split('-activation-')[-1].split('-')[0]
    hid_lay = pth.split('/')[-2].split('-num_hiddens-')[-1].split('-')[0]

    #algorithm = pth.split('/')[2] + '-' + dt
    #algorithm = act + '-' + hid_lay
    algorithm = "beta:" + pth.split('/')[-2].split('-beta-')[-1].split('-')[0]
    result = torch.load(pth)
    if algorithm in results and not is_one_model:
        start = len(results[algorithm])
        end = len(results[algorithm]) + len(result)
        j = 0
        for i in range(start, end):
            results[algorithm][i] = result[j]
            j += 1        
    elif algorithm in results and is_one_model:
        results[algorithm + "-" + str(seed)] = result
        seed += 1
    else:
        results[algorithm] = result
    

In [ ]:
results =OrderedDict(sorted(results.items(), key=lambda t: t[0]))
results.keys()

# Plot Error Curves
In this section we want to see if our Jacobian estimates with our neural network actually solve the undlerying task of interest (i.e. moving the end effector to some target position)

In [ ]:
def collect_eps_errs(result, dim=-1):
    err_over_time = []
    for k, value in result.items():
        episode_err = []
        for v in value:
            state = v[0]
            if dim < 0:
                psn = state[0:3]
                targ = state[-3:]
                mse = np.linalg.norm(targ - psn, 2)
                rmse = np.sqrt(mse)
                
                #mse = rmse
            else:
                psn = state[dim]
                targ = state[-3 + dim]
                mse = np.sqrt((targ - psn) ** 2)
            episode_err.append(mse)
        err_over_time.append(episode_err)
    return np.array(err_over_time).T

def plot_mu_sig(data, label=None, axis=1, ax=None):
    mean = np.array(data).mean(axis=axis)
    std = np.array(data).std(axis=axis)
    ste = std / np.sqrt(data.shape[1])
    
    if ax is None:
        plt.plot(mean, label=label)
        plt.fill_between(list(range(mean.shape[0])), mean + std, mean - std, alpha=0.1)
    else:
        ax.plot(mean, label=label)
        ax.fill_between(list(range(mean.shape[0])), mean + ste, mean - ste, alpha=0.1)

In [ ]:

err_curr_res = {}
for algorithm, result in results.items():
    err_over_time = collect_eps_errs(result, dim=-1)
    err_curr_res[algorithm] = err_over_time
    #plot_mu_sig(err_over_time, label=algorithm, ax=ax)
    
bounds = [0.0, 0.5, 1.0, 1.5, 2.0]
fig, axs = plt.subplots(len(bounds) - 1, 1, figsize=(15, 30))
i = 0
for start, end in zip(bounds[:-1], bounds[1:]):
    ax = axs[i]
    print(ax)
    for algorithm in results.keys():
        err_over_time = err_curr_res[algorithm]
        to_plot = np.logical_and(err_over_time[0, :] >= start, err_over_time[0, :] < end)
        plot_mu_sig(err_over_time[:, to_plot], label=algorithm +": {}".format(to_plot.sum()), ax=ax)
    
    
    ax.get_yaxis().set_ticks(np.linspace(0.0, end, 5))
    ax.legend()
    ax.set_title("start distance: {} - {}".format(start, end))
    if i == len(results):
        ax.set_xlabel("start mean squared error")
    else:
        ax.get_xaxis().set_ticks([])
    ax.set_xlabel("Time step")
    
    ax.set_ylabel("Error Decrease")
    i += 1

In [ ]:
titles = [str(i) for i in range(3)]
fig, axs = plt.subplots(len(titles), 1, figsize=(15, 25))
for i, ax in enumerate(axs):
    for algorithm, result in results.items():
        err_over_time = collect_eps_errs(result, dim=i)
        plot_mu_sig(err_over_time, label=algorithm, ax=ax)
        ax.set_title(titles[i])
        ax.legend()
        if i == 2:
            ax.set_xlabel("Time step")
        ax.set_ylabel("Error Decrease")

# Visualising start and end points for Trajectories

In this section we visualize the starting position against the end position. The intention of these plots is to understand how distance to the target might affect convergence

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15))

for algorithm, result in results.items():
    err_over_time = collect_eps_errs(result, dim=-1)
    
    start = err_over_time[0,:]
    end = err_over_time[-1,:]
    ax.scatter(start, end, label=algorithm)
    i += 1
    #ax.set_title(algorithm)
    ax.legend()
    ax.get_yaxis().set_ticks([0.0, 1.0,  2.0])
    ax.set_xlabel("start distance")
    ax.set_ylabel("end distance")

In [ ]:


fig, axs = plt.subplots(len(results), 1, figsize=(15, 15))
i = 0
for algorithm, result in results.items():
    ax = axs[i]
    err_over_time = collect_eps_errs(result, dim=-1)
    
    start = err_over_time[0,:]
    end = err_over_time[-1,:]
    ax.scatter(start, end, label=algorithm)
    i += 1
    #ax.set_title(algorithm)
    ax.legend()
    ax.get_yaxis().set_ticks([float(i)*0.1 for i in range(0, 20, 4)])
    if i == len(results):
        ax.set_xlabel("start mean squared error")
    else:
        ax.get_xaxis().set_ticks([])
    ax.set_ylabel("end mean squared error")

# Violin Plots of Area under the Curve

In this section we summarise the above error plots by looking at the AUC for each trajectory. Smaller AUC score are indicative of faster convergence to the target. This also gives us way to compare different algorithms a bit more 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 9))

labels = []
aucs = []
for algorithm, result in results.items():
    err_over_time = collect_eps_errs(result, dim=-1)
    #Here, we WANT to sum over each run
    #above we do not because that is less meaninful
    auc = err_over_time.sum(axis=0)   # for delta t between each timestep
    print(auc.shape)
    aucs.append(auc)
    labels.append(algorithm)
    
ax.violinplot(aucs, showmeans=False, showmedians=True)
ax.set_xticks(np.arange(1, len(labels) + 1))
ax.set_xticklabels(labels)
ax.yaxis.grid(True)
ax.set_ylabel("AUC of Trajectories")
ax.set_title("AUC of Trajectories for Different UVS Algorithms")

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(20, 20))

bounds = [0.0, 0.5, 1.0, 1.5, 2.0]
i = 0
violin_results = {}
for algorithm, result in results.items():
    err_over_time = collect_eps_errs(result, dim=-1)
    violin_results[algorithm] = err_over_time

for low, high in zip(bounds[:-1], bounds[1:]):
    labels = []
    aucs = []
    count = []
    for algorithm, result in violin_results.items():
        err_over_time = result#collect_eps_errs(result, dim=-1)
        #Here, we WANT to sum over each run
        #above we do not because that is less meaninful
        
        to_plot = np.logical_and(err_over_time[0, :] >= low, err_over_time[0, :] < high)
        count.append(to_plot.sum())
        
        to_plot = np.logical_and(to_plot, err_over_time[-1,:] <= .1)
        auc = err_over_time.sum(axis=0)   # for delta t between each timestep
        auc = auc[to_plot]
        if len(auc) == 0:
            auc = np.zeros(err_over_time.shape[-1])
        #print(auc.shape, low, high, to_plot.sum())
        aucs.append(auc)
        if "global-neuralnetwork" in algorithm:
            labels.append(algorithm.split("global-")[-1])
        else:
            labels.append(algorithm)
    ax = axs[i]
    i+= 1
    ax.violinplot(aucs, showmeans=False, showmedians=True)
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels([l + " {}/{}".format(len(a),c) for l, a, c in zip(labels, aucs,count)])
    ax.yaxis.grid(True)
    ax.set_ylabel("AUC of trajectory")
    ax.set_title("Initial distance: {} - {} meters".format(low, high))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 9))

labels = []
aucs = []
for algorithm, result in results.items():
    err_over_time = collect_eps_errs(result, dim=-1) 
    #want to sum over each trajectory
    auc = err_over_time.sum(axis=0)  #for delta t between each time step
    aucs.append(auc)
    labels.append(algorithm)
    
ax.boxplot(aucs, labels=labels)

ax.yaxis.grid(True)
ax.set_ylabel("AUC of Trajectories")
ax.set_title("AUC of Trajectories for Different UVS Algorithms")

# Comparing Jacobians

Here, we just calaculate the difference between Jacobians in trajectories to see how much they differ from the true underlying Jacobian

In [ ]:
def collect_jacobian_dist(result, norm=None):
    err_over_time = []
    for k, value in result.items():
        episode_err = []
        for v in value:
            estimate_J = v[-2] 

            true_J = v[-1]
            if norm is not "signs":
                mse = np.linalg.norm(estimate_J - true_J, norm) #, 'fro')
            else:
                sign_mismatch = np.logical_not(np.logical_and(estimate_J, true_J))
                mse = (sign_mismatch).sum() #techinically not mind you...
            episode_err.append(mse)
        err_over_time.append(episode_err)    
    
    return np.array(err_over_time).T

In [ ]:
#Focusing only on the frobenius norm
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

for algorithm, result in results.items():
    jac_err = collect_jacobian_dist(result)

    plot_mu_sig(jac_err, label=algorithm, axis=1, ax=ax)
    
ax.set_xlabel('Time step')
ax.set_ylabel('Jacobian distance')
ax.set_title("Frobenius Norm Distance")
ax.legend()


In [ ]:
#Focusing only on the frobenius norm
fig, ax = plt.subplots(3, 1, figsize=(15, 20))

activations = ['tanh', 'sigmoid', 'relu']

plot_to_use = {k: v for k,v in zip(activations, ax)}

for algorithm, result in results.items():
    act = algorithm.split('-')[0]
    jac_err = collect_jacobian_dist(result)
    ax = plot_to_use[act]
    plot_mu_sig(jac_err, label=algorithm, axis=1, ax=ax)
    
    ax.set_xlabel('Time step')
    ax.set_ylabel('Jacobian distance')
    ax.set_title("Frobenius Norm Distance")
    ax.legend()

In [ ]:
#Seeing if differen't norms highlight any othernotable differences
fig, axs = plt.subplots(4, 1, figsize=(10, 20))
norms = ['fro', 'nuc', np.inf, -np.inf]
for i, ax in enumerate(axs):
    n = norms[i]
    for algorithm, result in results.items():
        jac_err = collect_jacobian_dist(result, norm=n)
        plot_mu_sig(jac_err, label=algorithm, axis=1, ax=ax)
    if i == 3:
        ax.set_xlabel('Time step')
    ax.set_ylabel('Jacobian distance')
    ax.set_title(n)
    ax.legend()

In [ ]:
#Focusing only on the sign  norm
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

for algorithm, result in results.items():
    jac_err = collect_jacobian_dist(result, norm='signs')
    plot_mu_sig(jac_err, label=algorithm, axis=1, ax=ax)
    
ax.set_xlabel('Time step')
ax.set_ylabel('Jacobian distance')
ax.set_title("Frobenius Norm result")
ax.legend()

# Checking for Criteria of Convergence
The last thing I want to check for is for whether or not the critieria on J is met for the trajectories. I will need to review this to make sure I understand it, but if JJ^{+} has any non-positive eigen values (0 or less) then hypothetically, it should not converge.

I think hypothetically JJ^{+} should be the identity matrix...and if it's not then we have problems otherwise, I'll need to read up on that. 

In [ ]:
def check_pos_definite(A):
    eigs = np.linalg.eigvals(A)
            
    return (eigs > 0).all()
    
def check_jacobian_positive_definite(result, invert_true_J=False):
    err_over_time = []
    for k, value in result.items():
        episode_err = []
        for v in value:
            #Check against true jacobian
            #If our approximation is good, presumably the matrix should still be positive definite
            # i.e. the approximate J is transferable 
            J_hat = v[-2] 
            J = v[-1]
            
            if invert_true_J:
                iJ_hat = np.linalg.pinv(J_hat)
                JiJ = np.matmul(J, iJ_hat)
            else:
                iJ = np.linalg.pinv(J)
                JiJ = np.matmul(J_hat, iJ)
            is_pos_def = check_pos_definite(JiJ)
            loss = int(is_pos_def)
            episode_err.append(loss)
        err_over_time.append(episode_err)    
    
    return np.array(err_over_time).T


def check_jacobian_condition(result):
    conditioning_over_time = []
    true_solution_over_time = []
    for k, value in result.items():
        eps_conditioning = []
        true_conditioning = []
        for v in value:
            #Check against true jacobian
            #If our approximation is good, presumably the matrix should still be positive definite
            # i.e. the approximate J is transferable 
            #this was backwards.....
            J_hat = v[-1] 
            J = v[-2]
            
            #print("condition numbers:")
            #print("J hat", np.linalg.cond(J_hat), "J", np.linalg.cond(J), "JiJ", np.linalg.cond(JiJ))
            hat_cond = np.linalg.cond(J_hat)
            true_cond = np.linalg.cond(J)
            
            #iJ_hat = np.linalg.pinv(J_hat)
            #JiJ = np.matmul(J, iJ_hat)
            
            #JiJ_cond = np.linalg.cond(JiJ)
            eps_conditioning.append(hat_cond)
            true_conditioning.append(true_cond)
            
        conditioning_over_time.append(eps_conditioning)    
        true_solution_over_time.append(true_conditioning)
    return np.array(conditioning_over_time).T, np.array(true_solution_over_time).T

In [ ]:
# Checking conditioning of Matrix over episodes & comparing to true jacobian at those points
#Focusing only on the frobenius norm
fig, axs = plt.subplots(len(results.keys()), 1, figsize=(15, 20))

for i, k_v in enumerate(results.items()):
    algorithm, result = k_v[0], k_v[1]
    ax = axs[i]
    cond_hat, cond_true = check_jacobian_condition(result)
        
    plot_mu_sig(cond_hat, label="approximation", ax=ax)
    #plot_mu_sig(cond_true, label="true values", ax=ax)
    if i >= len(axs) - 1:
        ax.set_xlabel('Time step')
    ax.set_ylabel('Condition Number')
    ax.set_title("Checking Condition Number: {}".format(algorithm))
    ax.legend()

In [ ]:
#Focusing only on the frobenius norm
fig, axs = plt.subplots(len(results.keys()), 1, figsize=(15, 20))
#if not isinstance(axs, list):
#    axs = [axs]
for i, k_v in enumerate(results.items()):
    algorithm, result = k_v[0], k_v[1]
    ax = axs[i]
    jac_err = check_jacobian_positive_definite(result)
    
    #check if criteria is met for whole trajectory or not
    always_pos_def = (jac_err.sum(axis=0) >=200)
    not_always_pos_def = (jac_err.sum(axis=0) < 200)
    
    
    
    err_over_time = collect_eps_errs(result, dim=-1)
    traj_pos_def = err_over_time[:,always_pos_def]
    traj_not_pos_def = err_over_time[:, not_always_pos_def]
    
    
    total_traj = jac_err.shape[1]
    total_pos_d = always_pos_def.sum()
    total_not_pos_d = not_always_pos_def.sum()
    
    #plot_mu_sig(err_over_time, label="All {} Trajectories".format(total_traj), ax=ax)
    plot_mu_sig(traj_pos_def, label="+ Definite {} / {}".format(total_pos_d, total_traj), ax=ax)
    plot_mu_sig(traj_not_pos_def, label=" Not + Definite {}/{}".format(total_not_pos_d, total_traj), ax=ax)
    if i >= len(axs) - 1:
        ax.set_xlabel('Time step')
    ax.set_ylabel('MSE over time')
    ax.set_title("Positive Definite Criteria: {}".format(algorithm))
    ax.legend()

In [ ]:
#Focusing only on the frobenius norm
fig, axs = plt.subplots(len(results.keys()), 1, figsize=(15, 20))
#if not isinstance(axs, list):
#    axs = [axs]
for i, k_v in enumerate(results.items()):
    algorithm, result = k_v[0], k_v[1]
    ax = axs[i]
    jac_err = check_jacobian_positive_definite(result, True)
    
    #check if criteria is met for whole trajectory or not
    always_pos_def = (jac_err.sum(axis=0) >=100)
    not_always_pos_def = (jac_err.sum(axis=0) < 100)
    
    
    
    err_over_time = collect_eps_errs(result, dim=-1)
    traj_pos_def = err_over_time[:,always_pos_def]
    traj_not_pos_def = err_over_time[:, not_always_pos_def]
    
    
    total_traj = jac_err.shape[1]
    total_pos_d = always_pos_def.sum()
    total_not_pos_d = not_always_pos_def.sum()
    
    #plot_mu_sig(err_over_time, label="All {} Trajectories".format(total_traj), ax=ax)
    plot_mu_sig(traj_pos_def, label="+ Definite {} / {}".format(total_pos_d, total_traj), ax=ax)
    plot_mu_sig(traj_not_pos_def, label=" Not + Definite {}/{}".format(total_not_pos_d, total_traj), ax=ax)
    if i >= len(axs) - 1:
        ax.set_xlabel('Time step')
    ax.set_ylabel('MSE over time')
    ax.set_title("Positive Definite Criteria: {}".format(algorithm))
    ax.legend()

# Comparison of positive definiteness 
here we isolate to specific depths to see how results compare. Just makes it easier to see

In [ ]:
#Focusing only on the frobenius norm
fig, axs = plt.subplots(3, 1, figsize=(15, 20))
#if not isinstance(axs, list):
#    axs = [axs]
depth = str(8)
plot_indx = 0
for i, k_v in enumerate(results.items()):
    algorithm, result = k_v[0], k_v[1]
    if depth !=  algorithm.split('-')[1]:
        continue
    print(algorithm)
    ax = axs[plot_indx]
    plot_indx += 1
    jac_err = check_jacobian_positive_definite(result)
    
    #check if criteria is met for whole trajectory or not
    always_pos_def = (jac_err.sum(axis=0) >=100)
    not_always_pos_def = (jac_err.sum(axis=0) < 100)
    
    
    
    err_over_time = collect_eps_errs(result, dim=-1)
    traj_pos_def = err_over_time[:,always_pos_def]
    traj_not_pos_def = err_over_time[:, not_always_pos_def]
    
    
    total_traj = jac_err.shape[1]
    total_pos_d = always_pos_def.sum()
    total_not_pos_d = not_always_pos_def.sum()
    
    #plot_mu_sig(err_over_time, label="All {} Trajectories".format(total_traj), ax=ax)
    plot_mu_sig(traj_pos_def, label="+ Definite {} / {}".format(total_pos_d, total_traj), ax=ax)
    plot_mu_sig(traj_not_pos_def, label=" Not + Definite {}/{}".format(total_not_pos_d, total_traj), ax=ax)
    if i >= len(axs) - 1:
        ax.set_xlabel('Time step')
    ax.set_ylabel('MSE over time')
    ax.set_title("Positive Definite Criteria: {}".format(algorithm))
    ax.legend()

# Histogram view of condition number


In [ ]:
# Checking conditioning of Matrix over episodes & comparing to true jacobian at those points
#Focusing only on the frobenius norm
fig, axs = plt.subplots(len(results.keys()), 1, figsize=(15, 20))

for i, k_v in enumerate(results.items()):
    algorithm, result = k_v[0], k_v[1]
    ax = axs[i]
    cond_hat, cond_true = check_jacobian_condition(result)
        
    plot_mu_sig(cond_hat, label="approximation", ax=ax)
    #plot_mu_sig(cond_true, label="true values", ax=ax)
    if i >= len(axs) - 1:
        ax.set_xlabel('Time step')
    ax.set_ylabel('Condition Number')
    ax.set_title("Checking Condition Number: {}".format(algorithm))
    ax.legend()